# Assignment 1: Feature Extraction via Dimensionality Reduction using Autoencoders

## Problem Statement
This assignment focuses on feature extraction via dimensionality reduction using variants of autoencoders with CIFAR10 and MNIST datasets.

### Tasks Overview:
1. **Task 1**: PCA-based feature extraction and classification (Standard vs Randomized PCA)
2. **Task 2**: Single-layer linear autoencoder comparison with PCA
3. **Task 3**: Deep convolutional autoencoder with reconstruction error analysis
4. **Task 4**: MNIST 7-segment LED display classifier

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, constraints, regularizers
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

## Data Preparation
Load CIFAR10 dataset and convert to grayscale. Split into 70% training and 30% validation/test.

In [ ]:
# Load CIFAR10 dataset
# CIFAR10 contains 60,000 32x32 color images in 10 classes
# Default split: 50,000 training + 10,000 test
(X_train_color, y_train), (X_test_color, y_test) = cifar10.load_data()

print(f"Original CIFAR10 shapes:")
print(f"Training: {X_train_color.shape}, Labels: {y_train.shape}")
print(f"Test: {X_test_color.shape}, Labels: {y_test.shape}")

# Convert RGB images to grayscale as per assignment requirement
# TensorFlow's rgb_to_grayscale uses ITU-R 601-2 luma transform
# Formula: Gray = 0.299*R + 0.587*G + 0.114*B
# This reduces dimensionality from 3 channels to 1 while preserving luminance information
X_train_gray = tf.image.rgb_to_grayscale(X_train_color).numpy().squeeze()
X_test_gray = tf.image.rgb_to_grayscale(X_test_color).numpy().squeeze()

print(f"\nGrayscale shapes:")
print(f"Training: {X_train_gray.shape}")
print(f"Test: {X_test_gray.shape}")

# Combine and resplit data according to assignment requirement (70% train, 30% test)
# Original split was 50k/10k, we need 42k/18k
X_all = np.vstack([X_train_gray, X_test_gray])
y_all = np.vstack([y_train, y_test])

# Perform stratified split to maintain class distribution
# stratify=y_all ensures each split has same proportion of each class
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, train_size=0.7, random_state=42, stratify=y_all
)

print(f"\nFinal 70/30 split:")
print(f"Training: {X_train.shape}, Labels: {y_train.shape}")
print(f"Test: {X_test.shape}, Labels: {y_test.shape}")

# Flatten images for traditional ML methods (PCA, Logistic Regression)
# Convert from (N, 32, 32) to (N, 1024) for matrix operations
# Keep original shape for CNN-based autoencoders later
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

print(f"\nFlattened for PCA:")
print(f"Training: {X_train_flat.shape}")
print(f"Test: {X_test_flat.shape}")

# Flatten labels from (N, 1) to (N,) for sklearn compatibility
y_train = y_train.flatten()
y_test = y_test.flatten()

# Define class names for visualization and reporting
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Visualize sample grayscale images to verify conversion
# Display first 10 images with their class labels
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

for i in range(10):
    # Show grayscale image (single channel)
    axes[i].imshow(X_train[i], cmap='gray')
    axes[i].set_title(f"{class_names[y_train[i]]}")
    axes[i].axis('off')

plt.suptitle('Sample Grayscale CIFAR10 Images')
plt.tight_layout()
plt.show()

# Task 1: PCA-based Feature Extraction and Classification [2 marks]

### Objectives:
1. Perform standard PCA with 70% of training data
2. Identify K eigenvectors with 95% total energy
3. Train logistic regression classifier
4. Draw ROC curves for test dataset
5. Repeat with Randomized PCA and compare

In [ ]:
# Use 70% of training data for PCA as per assignment requirement
# This further splits the training set for PCA computation
n_samples_pca = int(0.7 * X_train_flat.shape[0])
X_pca_subset = X_train_flat[:n_samples_pca]

print(f"PCA training subset: {X_pca_subset.shape}")

# IMPORTANT: Normalize data AFTER splitting to prevent data leakage
# StandardScaler: centers data to mean=0 and scales to variance=1
# This is crucial for PCA as it's sensitive to feature scales
# fit_transform on training data learns mean and std
# transform on test data applies the same training statistics
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)
X_pca_subset_scaled = X_train_scaled[:n_samples_pca]

print(f"Data normalized: mean={X_train_scaled.mean():.4f}, std={X_train_scaled.std():.4f}")

### Standard PCA with 95% Energy Retention

In [ ]:
# Standard PCA with 95% variance (energy) retention
# n_components=0.95 means "select minimum K components that explain 95% variance"
# PCA finds orthogonal directions (principal components) that maximize variance
print("Performing Standard PCA...")
pca_standard = PCA(n_components=0.95, random_state=42)

# fit_transform: computes eigenvectors and projects training data
X_train_pca = pca_standard.fit_transform(X_train_scaled)

# transform: projects test data using already computed eigenvectors
X_test_pca = pca_standard.transform(X_test_scaled)

# K = number of components needed to retain 95% of total variance
K = pca_standard.n_components_
print(f"\nNumber of components (K) for 95% variance: {K}")
print(f"Explained variance ratio: {pca_standard.explained_variance_ratio_.sum():.4f}")
print(f"PCA transformed training shape: {X_train_pca.shape}")
print(f"PCA transformed test shape: {X_test_pca.shape}")

In [ ]:
# Plot cumulative explained variance to visualize PCA performance
# Cumulative variance shows how much information is retained as we add components
cumulative_variance = np.cumsum(pca_standard.explained_variance_ratio_)

plt.figure(figsize=(10, 5))
# Plot cumulative variance curve
plt.plot(range(1, len(cumulative_variance)+1), cumulative_variance, 'b-', linewidth=2)
# Mark 95% threshold (our target)
plt.axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
# Mark K components needed
plt.axvline(x=K, color='g', linestyle='--', label=f'K={K} components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Standard PCA: Cumulative Explained Variance')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

### Train Logistic Regression Classifier on PCA Features

In [ ]:
# Train logistic regression classifier on PCA-reduced features
# multi_class='multinomial': uses softmax for 10-class classification
# solver='lbfgs': Limited-memory BFGS optimizer (good for small datasets)
# n_jobs=-1: uses all CPU cores for parallel processing
print("Training Logistic Regression on Standard PCA features...")
lr_pca = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial',
                            solver='lbfgs', n_jobs=-1)
lr_pca.fit(X_train_pca, y_train)

# Evaluate model performance
train_acc_pca = lr_pca.score(X_train_pca, y_train)
test_acc_pca = lr_pca.score(X_test_pca, y_test)

print(f"\nStandard PCA + Logistic Regression Results:")
print(f"Training Accuracy: {train_acc_pca:.4f}")
print(f"Test Accuracy: {test_acc_pca:.4f}")

# Get predictions for further analysis (ROC curves, confusion matrix)
y_pred_pca = lr_pca.predict(X_test_pca)  # Class predictions
y_pred_proba_pca = lr_pca.predict_proba(X_test_pca)  # Probability scores for each class

print("\nClassification Report (Standard PCA):")
print(classification_report(y_test, y_pred_pca, target_names=class_names))

### ROC Curves for Standard PCA (One-vs-Rest)

In [ ]:
# Prepare data for multi-class ROC curves using One-vs-Rest strategy
# Binarize labels: convert single label (0-9) to 10 binary columns
# Example: label 3 becomes [0,0,0,1,0,0,0,0,0,0]
y_test_bin = label_binarize(y_test, classes=np.arange(10))
n_classes = y_test_bin.shape[1]

# Compute ROC curve and AUC (Area Under Curve) for each class
# ROC curve plots True Positive Rate vs False Positive Rate
# AUC closer to 1.0 indicates better classification
fpr_pca = dict()  # False Positive Rate
tpr_pca = dict()  # True Positive Rate
roc_auc_pca = dict()  # Area Under Curve

for i in range(n_classes):
    # For each class, treat it as binary: this class vs all others
    fpr_pca[i], tpr_pca[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba_pca[:, i])
    roc_auc_pca[i] = auc(fpr_pca[i], tpr_pca[i])

# Plot ROC curves for all 10 classes
plt.figure(figsize=(12, 8))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for i, color in enumerate(colors):
    plt.plot(fpr_pca[i], tpr_pca[i], color=color, lw=2,
             label=f'{class_names[i]} (AUC = {roc_auc_pca[i]:.3f})')

# Diagonal line represents random classifier (AUC = 0.5)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Standard PCA + Logistic Regression', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nMean AUC (Standard PCA): {np.mean(list(roc_auc_pca.values())):.4f}")

### Randomized PCA Implementation

In [ ]:
# Randomized PCA: Faster approximation using randomized SVD
# Uses random projections for faster computation on large datasets
# Particularly useful when K << n_features
# Should give similar results to standard PCA with much faster computation
print(f"Performing Randomized PCA with K={K} components...")
pca_randomized = PCA(n_components=K, svd_solver='randomized', random_state=42)
X_train_pca_rand = pca_randomized.fit_transform(X_train_scaled)
X_test_pca_rand = pca_randomized.transform(X_test_scaled)

print(f"\nRandomized PCA Results:")
print(f"Number of components: {pca_randomized.n_components_}")
print(f"Explained variance ratio: {pca_randomized.explained_variance_ratio_.sum():.4f}")
print(f"Transformed training shape: {X_train_pca_rand.shape}")

In [ ]:
# Train logistic regression on Randomized PCA features
# Using same hyperparameters as standard PCA for fair comparison
print("Training Logistic Regression on Randomized PCA features...")
lr_pca_rand = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial',
                                 solver='lbfgs', n_jobs=-1)
lr_pca_rand.fit(X_train_pca_rand, y_train)

# Evaluate model performance
train_acc_pca_rand = lr_pca_rand.score(X_train_pca_rand, y_train)
test_acc_pca_rand = lr_pca_rand.score(X_test_pca_rand, y_test)

print(f"\nRandomized PCA + Logistic Regression Results:")
print(f"Training Accuracy: {train_acc_pca_rand:.4f}")
print(f"Test Accuracy: {test_acc_pca_rand:.4f}")

# Generate predictions for ROC analysis
y_pred_pca_rand = lr_pca_rand.predict(X_test_pca_rand)
y_pred_proba_pca_rand = lr_pca_rand.predict_proba(X_test_pca_rand)

print("\nClassification Report (Randomized PCA):")
print(classification_report(y_test, y_pred_pca_rand, target_names=class_names))

### ROC Curves for Randomized PCA

In [ ]:
# Compute ROC curves for Randomized PCA classifier
# Same methodology as standard PCA for direct comparison
fpr_pca_rand = dict()
tpr_pca_rand = dict()
roc_auc_pca_rand = dict()

for i in range(n_classes):
    # One-vs-Rest ROC for each class
    fpr_pca_rand[i], tpr_pca_rand[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba_pca_rand[:, i])
    roc_auc_pca_rand[i] = auc(fpr_pca_rand[i], tpr_pca_rand[i])

# Plot ROC curves for all classes
plt.figure(figsize=(12, 8))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for i, color in enumerate(colors):
    plt.plot(fpr_pca_rand[i], tpr_pca_rand[i], color=color, lw=2,
             label=f'{class_names[i]} (AUC = {roc_auc_pca_rand[i]:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Randomized PCA + Logistic Regression', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nMean AUC (Randomized PCA): {np.mean(list(roc_auc_pca_rand.values())):.4f}")

In [ ]:
# Direct ROC curve comparison: Standard PCA vs Randomized PCA
# Compare the average performance across all classes

# Calculate mean ROC curve for Standard PCA
# Interpolate all class ROC curves to common FPR points for averaging
mean_fpr = np.linspace(0, 1, 100)
tprs_std = []
for i in range(n_classes):
    tprs_std.append(np.interp(mean_fpr, fpr_pca[i], tpr_pca[i]))
    tprs_std[-1][0] = 0.0
mean_tpr_std = np.mean(tprs_std, axis=0)
mean_tpr_std[-1] = 1.0
mean_auc_std = auc(mean_fpr, mean_tpr_std)

# Calculate mean ROC curve for Randomized PCA
tprs_rand = []
for i in range(n_classes):
    tprs_rand.append(np.interp(mean_fpr, fpr_pca_rand[i], tpr_pca_rand[i]))
    tprs_rand[-1][0] = 0.0
mean_tpr_rand = np.mean(tprs_rand, axis=0)
mean_tpr_rand[-1] = 1.0
mean_auc_rand = auc(mean_fpr, mean_tpr_rand)

# Plot comparison
plt.figure(figsize=(10, 7))
plt.plot(mean_fpr, mean_tpr_std,
         label=f'Standard PCA (AUC = {mean_auc_std:.4f})',
         color='blue', linewidth=2.5)
plt.plot(mean_fpr, mean_tpr_rand,
         label=f'Randomized PCA (AUC = {mean_auc_rand:.4f})',
         color='red', linewidth=2.5, linestyle='--')
plt.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.3, label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve Comparison: Standard vs Randomized PCA\n(Average across all classes)',
         fontsize=13, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary
print("\nROC Comparison Summary:")
print(f"Standard PCA Mean AUC:    {mean_auc_std:.4f}")
print(f"Randomized PCA Mean AUC:  {mean_auc_rand:.4f}")
print(f"Difference:               {abs(mean_auc_std - mean_auc_rand):.4f}")
print("\nConclusion: Both methods achieve nearly identical performance.")

### Comparison: Standard vs Randomized PCA

In [ ]:
# Compare Standard vs Randomized PCA performance
# Both methods should give very similar results
# Randomized PCA is faster but uses approximation
comparison_data = {
    'Metric': ['Components (K)', 'Explained Variance', 'Training Accuracy', 'Test Accuracy', 'Mean AUC'],
    'Standard PCA': [
        K,
        f"{pca_standard.explained_variance_ratio_.sum():.4f}",
        f"{train_acc_pca:.4f}",
        f"{test_acc_pca:.4f}",
        f"{np.mean(list(roc_auc_pca.values())):.4f}"
    ],
    'Randomized PCA': [
        K,
        f"{pca_randomized.explained_variance_ratio_.sum():.4f}",
        f"{train_acc_pca_rand:.4f}",
        f"{test_acc_pca_rand:.4f}",
        f"{np.mean(list(roc_auc_pca_rand.values())):.4f}"
    ]
}

import pandas as pd
comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*60)
print("COMPARISON: Standard PCA vs Randomized PCA")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)

# Task 2: Single-Layer Linear Autoencoder vs PCA [2 marks]

### Objectives:
1. Train single-layer autoencoder with K encoder nodes
2. Linear activation function
3. Mean and variance normalized input
4. Constraint: decoder weights = encoder weights transpose
5. Unit magnitude weight vectors
6. Compare eigenvectors from PCA with autoencoder weights
7. Display both as grayscale images

In [ ]:
# Custom Model for Tied Weights Linear Autoencoder
# This implementation satisfies all assignment requirements for Task 2

class TiedLinearAutoencoder(tf.keras.Model):
    """
    Linear autoencoder with tied weights and unit norm constraint.

    Architecture:
        Encoder:  h = X @ W  (where W has unit-norm columns)
        Decoder:  X' = h @ W^T

    Key Properties (Assignment Requirements):
        1. Decoder weights = Encoder weights transpose (tied weights)
        2. Each weight vector has unit magnitude (UnitNorm constraint)
        3. Linear activation throughout (no non-linearity)
        4. No bias terms (data is pre-centered with StandardScaler)
        5. Theoretically equivalent to PCA for centered data

    Parameters:
        input_dim: Dimensionality of input (1024 for 32x32 grayscale images)
        latent_dim: Dimensionality of latent space (K from PCA)
    """
    def __init__(self, input_dim, latent_dim):
        super(TiedLinearAutoencoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # ENCODER WEIGHT MATRIX W: Shape (input_dim, latent_dim)
        # This is the ONLY trainable parameter in the model
        #
        # Constraint: UnitNorm(axis=0) enforces unit magnitude per column
        # - axis=0 means normalization along rows (each column becomes unit vector)
        # - Each column W[:, i] represents one latent dimension's basis vector
        # - After constraint: ||W[:, i]||_2 = 1 for all i
        #
        # This constraint makes the autoencoder theoretically equivalent to PCA:
        # - PCA eigenvectors also have unit norm
        # - Both minimize reconstruction error via linear projection
        self.W = self.add_weight(
            shape=(input_dim, latent_dim),
            initializer="glorot_uniform",  # Xavier uniform initialization
            constraint=tf.keras.constraints.UnitNorm(axis=0),  # Unit norm per column
            trainable=True,
            name="encoder_weights"
        )

    def call(self, inputs):
        """
        Forward pass through the autoencoder.

        Args:
            inputs: Input data, shape (batch_size, input_dim)
                    Assumed to be centered (mean=0) via StandardScaler

        Returns:
            reconstruction: Reconstructed data, shape (batch_size, input_dim)

        Mathematics:
            Encoder:  h = X @ W
                      Maps input to latent space (1024 -> K dimensions)

            Decoder:  X' = h @ W^T = (X @ W) @ W^T
                      Reconstructs input from latent representation
                      Uses TRANSPOSED encoder weights (tied weights)
        """
        # ENCODER: Project input to latent space
        # Linear transformation without bias (data is centered)
        # Shape: (batch_size, input_dim) @ (input_dim, latent_dim) -> (batch_size, latent_dim)
        hidden = tf.matmul(inputs, self.W)

        # DECODER: Reconstruct input from latent representation
        # Uses transposed encoder weights (tied weights constraint)
        # Shape: (batch_size, latent_dim) @ (latent_dim, input_dim) -> (batch_size, input_dim)
        reconstruction = tf.matmul(hidden, tf.transpose(self.W))

        return reconstruction

In [ ]:
# Build and compile the tied weights linear autoencoder
input_dim = X_train_flat.shape[1]  # 32*32 = 1024 pixels per grayscale image
encoding_dim = K  # Use K from PCA (number of components for 95% variance retention)

print(f"Building Linear Autoencoder with {encoding_dim} latent dimensions...")

# Instantiate the custom autoencoder model
# This creates the weight matrix W with shape (1024, K)
autoencoder_linear = TiedLinearAutoencoder(input_dim, encoding_dim)

# Compile the model with appropriate loss and metrics
# Loss: MSE (Mean Squared Error) - measures reconstruction quality
#   - Minimizing MSE is equivalent to maximizing explained variance
#   - This is the SAME objective as PCA (hence theoretical equivalence)
# Metrics: MAE (Mean Absolute Error) - alternative reconstruction metric
autoencoder_linear.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Build the model by calling it once (Keras requirement for model summary)
# This initializes all weights and establishes the computation graph
_ = autoencoder_linear(X_train_scaled[:1])

# Display model information
print("\nModel Architecture:")
print(f"  Input dimension:  {input_dim}")
print(f"  Latent dimension: {encoding_dim}")
print(f"  Encoder weights:  {input_dim} x {encoding_dim} (unit norm columns)")
print(f"  Decoder weights:  {encoding_dim} x {input_dim} (transpose of encoder)")
print(f"  Biases:           None (data is centered via StandardScaler)")
print(f"  Total parameters: {sum([tf.size(w).numpy() for w in autoencoder_linear.trainable_weights]):,}")
print(f"\nNote: No bias terms are needed because:")
print(f"  1. Input data is standardized (mean=0, std=1)")
print(f"  2. This ensures theoretical equivalence with PCA")
print(f"  3. PCA also assumes centered data with no bias terms")

In [ ]:
# Train the linear autoencoder
# Training on normalized data (same preprocessing as PCA)
# Objective: Minimize reconstruction error (MSE)
print("Training Linear Autoencoder...")

history_linear = autoencoder_linear.fit(
    X_train_scaled, X_train_scaled,  # Input = target for autoencoder
    epochs=50,  # May need more epochs than CNN due to constraints
    batch_size=256,
    shuffle=True,  # Randomize training order
    validation_data=(X_test_scaled, X_test_scaled),
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Plot training history to monitor convergence
# Loss should decrease smoothly if training is stable
# Validation loss close to training loss indicates no overfitting
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# MSE Loss over epochs
axes[0].plot(history_linear.history['loss'], label='Training Loss')
axes[0].plot(history_linear.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Linear Autoencoder - Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE over epochs (alternative metric)
axes[1].plot(history_linear.history['mae'], label='Training MAE')
axes[1].plot(history_linear.history['val_mae'], label='Validation MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Linear Autoencoder - MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Extract and Compare Eigenvectors (PCA) vs Weight Vectors (Autoencoder)

In [ ]:
# Extract eigenvectors and weight matrices for comparison
# PCA components: directions of maximum variance
# Autoencoder weights: learned feature detectors
pca_components = pca_standard.components_  # Shape: (K, 1024)

# Extract autoencoder weight matrix W from the model
ae_weights = autoencoder_linear.W.numpy()  # Shape: (1024, K)
ae_components = ae_weights.T  # Transpose to match PCA format: (K, 1024)

print(f"PCA components shape: {pca_components.shape}")
print(f"Autoencoder weight matrix shape: {ae_components.shape}")

# Both can be reshaped to 32x32 for visualization
img_shape = (32, 32)

In [ ]:
# Visualize PCA eigenvectors as grayscale images
# Each eigenvector captures a pattern of variance in the data
n_display = min(16, K)
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for i in range(n_display):
    # PCA components are stored as rows, reshape to 32x32 image
    eigenvector = pca_components[i].reshape(img_shape)
    axes[i].imshow(eigenvector, cmap='gray')
    axes[i].set_title(f'PCA {i+1}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('PCA Eigenvectors (Top 16)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize autoencoder weight vectors as grayscale images
# These should be similar to PCA eigenvectors (theoretical equivalence)
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for i in range(n_display):
    # Autoencoder weights transposed to match PCA format, reshape to 32x32
    weight_vector = ae_components[i].reshape(img_shape)
    axes[i].imshow(weight_vector, cmap='gray')
    axes[i].set_title(f'AE {i+1}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Autoencoder Weight Vectors (Top 16)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Side-by-side comparison: PCA vs Autoencoder
# Direct visual comparison to see structural similarities
n_compare = min(10, K)  # Show 10 pairs for clearer comparison

fig, axes = plt.subplots(2, n_compare, figsize=(20, 4))

# Top row: PCA eigenvectors
for i in range(n_compare):
    axes[0, i].imshow(pca_components[i].reshape(img_shape), cmap='gray')
    axes[0, i].set_title(f'PCA {i+1}', fontsize=9)
    axes[0, i].axis('off')

# Bottom row: Autoencoder weights
for i in range(n_compare):
    axes[1, i].imshow(ae_components[i].reshape(img_shape), cmap='gray')
    axes[1, i].set_title(f'AE {i+1}', fontsize=9)
    axes[1, i].axis('off')

# Add row labels on the left
fig.text(0.02, 0.75, 'PCA', fontsize=12, fontweight='bold', va='center', rotation=90)
fig.text(0.02, 0.25, 'Autoencoder', fontsize=12, fontweight='bold', va='center', rotation=90)

plt.suptitle('Comparison: PCA Eigenvectors vs Autoencoder Weights',
             fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0.03, 0, 1, 0.96])
plt.show()

### Commentary on PCA vs Autoencoder Comparison

**Observations:**

1. **Structural Similarity**: Both PCA eigenvectors and autoencoder weight vectors capture similar low-frequency patterns and principal directions in the data. This is expected since both methods perform linear dimensionality reduction optimizing for reconstruction.

2. **Theoretical Equivalence**: For a linear autoencoder with MSE loss and tied weights (decoder = encoder transpose), the learned weight vectors span the same subspace as the top K PCA eigenvectors. However, they may differ in:
   - **Ordering**: PCA components are sorted by explained variance, while autoencoder weights have no inherent order
   - **Sign**: Eigenvectors can be multiplied by -1 without changing the solution
   - **Orthogonality**: PCA guarantees orthogonal components, autoencoder does not

3. **Visual Patterns**: 
   - Early components capture global structure (edges, orientations)
   - Later components capture finer details and noise
   - Both show grayscale patterns representing feature detectors learned from data

4. **Practical Differences**:
   - PCA is deterministic and finds global optimum
   - Autoencoder optimization is iterative (gradient descent) and may converge to local optima
   - PCA requires computing full covariance matrix (memory intensive for large data)
   - Autoencoders can be trained with mini-batch SGD (scalable)

# Task 3: Deep Convolutional Autoencoders [4 marks]

### Objectives:
1. Design deep convolutional autoencoder with latent dimension ≈ K
2. Calculate reconstruction error on test set
3. Compare with single hidden layer K-node autoencoder (sigmoid)
4. Build 3-hidden-layer variant with K nodes distributed equally
5. Compare reconstruction errors across all variants

In [ ]:
# Prepare data for convolutional autoencoders
# CNNs expect 4D input: (batch_size, height, width, channels)
# Normalize to [0, 1] range for sigmoid output activation compatibility
X_train_cnn = X_train.reshape(-1, 32, 32, 1) / 255.0  # Add channel dimension
X_test_cnn = X_test.reshape(-1, 32, 32, 1) / 255.0

print(f"CNN input shapes:")
print(f"Training: {X_train_cnn.shape}")
print(f"Test: {X_test_cnn.shape}")
print(f"Value range: [{X_train_cnn.min():.3f}, {X_train_cnn.max():.3f}]")

### Model 1: Deep Convolutional Autoencoder

In [ ]:
# Deep Convolutional Autoencoder architecture
# Uses convolutional layers to learn hierarchical spatial features
def build_deep_conv_autoencoder(latent_dim):
    """
    Architecture Overview:
        ENCODER:  32x32x1 -> 16x16x32 -> 8x8x64 -> 4x4x128 -> 2048 -> K
        DECODER:  K -> 2048 -> 4x4x128 -> 8x8x128 -> 16x16x64 -> 32x32x32 -> 32x32x1
    """

    # ========================================================================
    # ENCODER: Progressive downsampling (shrinking spatial dimensions)
    # ========================================================================
    encoder_input = layers.Input(shape=(32, 32, 1))

    # --- LAYER 1: Extract low-level features (edges, textures) ---
    # Conv2D: Applies 32 filters of size 3x3 to detect patterns
    #   - filters=32: learns 32 different feature detectors
    #   - kernel_size=(3,3): each filter looks at 3x3 pixel neighborhoods
    #   - activation='relu': introduces non-linearity, helps learn complex patterns
    #   - padding='same': maintains spatial dimensions (32x32 stays 32x32)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)

    # MaxPooling2D: Reduces spatial dimensions by taking maximum value in 2x2 windows
    #   - pool_size=(2,2): divides height and width by 2
    #   - padding='same': handles odd dimensions gracefully
    #   - Result: 32x32x32 -> 16x16x32 (spatial shrinking begins)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 32x32 -> 16x16

    # --- LAYER 2: Extract mid-level features (shapes, patterns) ---
    # More filters (64) to capture increased complexity at lower resolution
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 16x16 -> 8x8

    # --- LAYER 3: Extract high-level features (object parts) ---
    # Even more filters (128) for abstract representations
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 8x8 -> 4x4

    # ========================================================================
    # LATENT SPACE: Bottleneck representation (maximum compression)
    # ========================================================================
    # Flatten: Converts 3D spatial data to 1D vector
    #   - Input: 4x4x128 spatial feature maps
    #   - Output: 2048-dimensional vector (4 * 4 * 128 = 2048)
    #   - This destroys spatial structure but preserves all information
    x = layers.Flatten()(x)  # 4x4x128 = 2048 features

    # Dense: Compresses to K-dimensional latent representation
    #   - 2048 -> latent_dim (K): Maximum information compression
    #   - This K-dim vector encodes the entire 32x32 image
    #   - ReLU activation maintains non-negative activations
    latent = layers.Dense(latent_dim, activation='relu', name='latent')(x)

    # ========================================================================
    # DECODER: Progressive upsampling (expanding to reconstruct input)
    # ========================================================================
    # Dense: Expand from K dimensions back to spatial representation size
    #   - latent_dim (K) -> 2048: First expansion step
    #   - Prepares to reshape back into 4x4x128 spatial format
    x = layers.Dense(4 * 4 * 128, activation='relu')(latent)

    # Reshape: Convert 1D vector back to 3D spatial format
    #   - 2048 -> 4x4x128: Restores spatial structure
    #   - Starting point for spatial upsampling
    x = layers.Reshape((4, 4, 128))(x)  # Reshape back to spatial format

    # --- Upsampling path (mirror of encoder, expands spatial dimensions) ---

    # Conv2DTranspose: Learns to upsample while refining features
    #   - Also called "deconvolution" or "transposed convolution"
    #   - strides=2: doubles spatial dimensions (opposite of pooling)
    #   - Learns HOW to upsample (unlike simple interpolation)
    #   - padding='same': ensures output size is exactly stride * input_size

    # Layer 1 decoder: 4x4x128 -> 8x8x128
    #   - 128 filters: maintains feature richness while upsampling
    #   - strides=2: 4x4 -> 8x8 (doubles height and width)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation='relu', padding='same')(x)

    # Layer 2 decoder: 8x8x128 -> 16x16x64
    #   - 64 filters: reduces feature count as we approach output
    #   - strides=2: 8x8 -> 16x16 (doubles again)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same')(x)

    # Layer 3 decoder: 16x16x64 -> 32x32x32
    #   - 32 filters: further reduces feature count
    #   - strides=2: 16x16 -> 32x32 (final spatial expansion)
    x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation='relu', padding='same')(x)

    # ========================================================================
    # OUTPUT LAYER: Reconstruct single-channel grayscale image
    # ========================================================================
    # Final Conv2D: 32x32x32 -> 32x32x1
    #   - filters=1: reduces to single channel (grayscale)
    #   - activation='sigmoid': outputs values in [0, 1] range
    #   - padding='same': maintains 32x32 spatial dimensions
    #   - Sigmoid ensures output matches normalized input range
    decoder_output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(encoder_input, decoder_output, name='deep_conv_autoencoder')
    return autoencoder

# Build model with latent dimension = K (from PCA)
conv_ae = build_deep_conv_autoencoder(latent_dim=K)
conv_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(f"\nDeep Convolutional Autoencoder (Latent Dim = {K})")
conv_ae.summary()

In [ ]:
# Train deep convolutional autoencoder
# This should learn hierarchical spatial features
# Expected to outperform simple linear/fully-connected autoencoders
print("Training Deep Convolutional Autoencoder...")

history_conv = conv_ae.fit(
    X_train_cnn, X_train_cnn,  # Autoencoder: input = target
    epochs=30,  # CNNs typically need fewer epochs due to strong inductive bias
    batch_size=128,  # Smaller batch for better gradient estimates
    shuffle=True,
    validation_data=(X_test_cnn, X_test_cnn),
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Evaluate reconstruction error on test set
# Lower MSE indicates better reconstruction quality
# Use test set for fair comparison (unseen data)
X_test_reconstructed_conv = conv_ae.predict(X_test_cnn)
reconstruction_error_conv = np.mean((X_test_cnn - X_test_reconstructed_conv) ** 2)

print(f"\nDeep Convolutional Autoencoder:")
print(f"Test Reconstruction Error (MSE): {reconstruction_error_conv:.6f}")
print(f"Test Reconstruction Error (RMSE): {np.sqrt(reconstruction_error_conv):.6f}")

### Model 2: Single Hidden Layer Autoencoder (Sigmoid Activation)

In [ ]:
# Single hidden layer autoencoder with sigmoid activation
# Simpler architecture than convolutional - uses fully connected layers
# Sigmoid activation introduces non-linearity (unlike linear autoencoder in Task 2)
def build_single_layer_ae(latent_dim):
    """
    Architecture Overview:
        ENCODER:  32x32x1 -> [Flatten] -> 1024 -> K (sigmoid)
        DECODER:  K -> 1024 (linear) -> [Reshape] -> 32x32x1

    Key difference from Conv AE:
        - No spatial structure preservation (flatten immediately)
        - Single compression step (1024 -> K)
        - Fully connected layers (every input connects to every output)
    """

    # ========================================================================
    # ENCODER: Single-step compression with sigmoid non-linearity
    # ========================================================================
    encoder_input = layers.Input(shape=(32, 32, 1))

    # Flatten: Convert 2D image to 1D vector
    #   - Input: 32x32x1 spatial image
    #   - Output: 1024-dimensional vector (32 * 32 * 1 = 1024)
    #   - Destroys spatial structure immediately (unlike Conv AE)
    #   - Every pixel becomes an independent feature
    x = layers.Flatten()(encoder_input)  # 32x32x1 -> 1024

    # Dense layer with sigmoid: Non-linear compression to latent space
    #   - Input: 1024 features
    #   - Output: latent_dim (K) features
    #   - activation='sigmoid': squashes values to [0, 1] range
    #   - Creates non-linear bottleneck (unlike linear autoencoder in Task 2)
    #   - Each of K neurons learns a weighted combination of all 1024 pixels
    #   - Parameters: 1024 * K weights + K biases
    latent = layers.Dense(latent_dim, activation='sigmoid', name='latent')(x)

    # ========================================================================
    # DECODER: Single-step expansion with linear activation
    # ========================================================================
    # Dense layer with linear activation: Reconstruct pixel values
    #   - Input: latent_dim (K) features
    #   - Output: 1024 pixel values
    #   - activation='linear': no constraint on output range
    #   - Allows reconstruction of full pixel intensity range
    #   - Parameters: K * 1024 weights + 1024 biases
    x = layers.Dense(32 * 32, activation='linear')(latent)

    # Reshape: Convert 1D vector back to 2D image
    #   - Input: 1024-dimensional vector
    #   - Output: 32x32x1 spatial image
    #   - Restores spatial structure for visualization and comparison
    decoder_output = layers.Reshape((32, 32, 1))(x)

    autoencoder = models.Model(encoder_input, decoder_output, name='single_layer_ae')
    return autoencoder

single_ae = build_single_layer_ae(latent_dim=K)
single_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(f"\nSingle Hidden Layer Autoencoder (K={K} nodes, Sigmoid)")
single_ae.summary()

In [ ]:
# Train single layer autoencoder
# Compare with conv autoencoder to see benefit of spatial structure
print("Training Single Hidden Layer Autoencoder...")

history_single = single_ae.fit(
    X_train_cnn, X_train_cnn,
    epochs=30,  # Same as conv AE for fair comparison
    batch_size=128,
    shuffle=True,
    validation_data=(X_test_cnn, X_test_cnn),
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Evaluate reconstruction error for single layer autoencoder
# Expected to be worse than conv AE but better than linear AE
X_test_reconstructed_single = single_ae.predict(X_test_cnn)
reconstruction_error_single = np.mean((X_test_cnn - X_test_reconstructed_single) ** 2)

print(f"\nSingle Hidden Layer Autoencoder:")
print(f"Test Reconstruction Error (MSE): {reconstruction_error_single:.6f}")
print(f"Test Reconstruction Error (RMSE): {np.sqrt(reconstruction_error_single):.6f}")

### Model 3: Three Hidden Layer Autoencoder (K nodes distributed equally)

In [ ]:
# Three hidden layer autoencoder with K nodes distributed equally
# Tests if distributing capacity across layers improves performance
# Architecture: gradually compress then expand (hourglass/symmetric shape)
def build_three_layer_ae(total_nodes):
    """
    Architecture Overview:
        ENCODER:  32x32x1 -> [Flatten] -> 1024 -> K*3 -> K*2 -> K (sigmoid at each layer)
        DECODER:  K -> K*2 -> K*3 -> 1024 (linear) -> [Reshape] -> 32x32x1

    Key difference from Single Layer AE:
        - Gradual compression (3 steps vs 1 step)
        - Hierarchical feature learning (like Conv AE but without spatial structure)
        - Total capacity distributed across 3 bottleneck layers
    """
    nodes_per_layer = total_nodes // 3  # Distribute K nodes equally

    # ========================================================================
    # ENCODER: Multi-step compression (gradual bottleneck)
    # ========================================================================
    encoder_input = layers.Input(shape=(32, 32, 1))

    # Flatten: Convert 2D image to 1D vector
    #   - Input: 32x32x1 spatial image
    #   - Output: 1024-dimensional vector
    x = layers.Flatten()(encoder_input)  # 32x32x1 -> 1024

    # Layer 1: First compression step (1024 -> K*3)
    #   - Mild compression: reduces to ~3x the final bottleneck size
    #   - activation='sigmoid': non-linear transformation [0, 1]
    #   - Learns first level of abstract features
    #   - Example: If K=160, this layer has 480 nodes (160*3)
    x = layers.Dense(nodes_per_layer * 3, activation='sigmoid')(x)  # First compression

    # Layer 2: Second compression step (K*3 -> K*2)
    #   - Further compression: reduces to ~2x the final bottleneck size
    #   - activation='sigmoid': continues non-linear feature extraction
    #   - Learns second level of more abstract features
    #   - Example: If K=160, this layer has 320 nodes (160*2)
    x = layers.Dense(nodes_per_layer * 2, activation='sigmoid')(x)  # Further compress

    # Layer 3: Final compression step (K*2 -> K)
    #   - Maximum compression: reaches the bottleneck
    #   - activation='sigmoid': final non-linear encoding
    #   - Creates the K-dimensional latent representation
    #   - Example: If K=160, this layer has 160 nodes (160*1)
    latent = layers.Dense(nodes_per_layer, activation='sigmoid', name='latent')(x)  # Bottleneck

    # ========================================================================
    # DECODER: Multi-step expansion (gradual reconstruction)
    # ========================================================================
    # Mirror structure of encoder (symmetric hourglass shape)

    # Layer 1: First expansion step (K -> K*2)
    #   - Begin reconstruction: doubles the representation size
    #   - activation='sigmoid': maintains non-linearity
    #   - Starts expanding abstract features back toward pixel space
    x = layers.Dense(nodes_per_layer * 2, activation='sigmoid')(latent)

    # Layer 2: Second expansion step (K*2 -> K*3)
    #   - Continue expansion: 1.5x increase in size
    #   - activation='sigmoid': continues feature refinement
    #   - Further expands toward full pixel representation
    x = layers.Dense(nodes_per_layer * 3, activation='sigmoid')(x)

    # Layer 3: Final expansion step (K*3 -> 1024)
    #   - Full expansion: back to original 1024 pixel values
    #   - activation='linear': no constraint on output range (allows full reconstruction)
    #   - Directly outputs pixel intensities
    x = layers.Dense(32 * 32, activation='linear')(x)  # Linear at final layer

    # Reshape: Convert 1D vector back to 2D image
    #   - Input: 1024-dimensional vector
    #   - Output: 32x32x1 spatial image
    decoder_output = layers.Reshape((32, 32, 1))(x)

    autoencoder = models.Model(encoder_input, decoder_output, name='three_layer_ae')
    return autoencoder, nodes_per_layer

three_layer_ae, nodes_per_layer = build_three_layer_ae(total_nodes=K)
three_layer_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(f"\nThree Hidden Layer Autoencoder (≈{nodes_per_layer} nodes per layer)")
print(f"Architecture: 1024 -> {nodes_per_layer*3} -> {nodes_per_layer*2} -> {nodes_per_layer} -> {nodes_per_layer*2} -> {nodes_per_layer*3} -> 1024")
three_layer_ae.summary()

In [ ]:
# Train three layer autoencoder
# Deeper network may learn better hierarchical features
print("Training Three Hidden Layer Autoencoder...")

history_three = three_layer_ae.fit(
    X_train_cnn, X_train_cnn,
    epochs=30,
    batch_size=128,
    shuffle=True,
    validation_data=(X_test_cnn, X_test_cnn),
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Evaluate reconstruction error for three layer autoencoder
# Compare: Does distributing nodes help vs single layer?
X_test_reconstructed_three = three_layer_ae.predict(X_test_cnn)
reconstruction_error_three = np.mean((X_test_cnn - X_test_reconstructed_three) ** 2)

print(f"\nThree Hidden Layer Autoencoder:")
print(f"Test Reconstruction Error (MSE): {reconstruction_error_three:.6f}")
print(f"Test Reconstruction Error (RMSE): {np.sqrt(reconstruction_error_three):.6f}")

### Visualize Reconstructions

In [ ]:
# Visualize original vs reconstructed images for all 3 autoencoders
# This allows qualitative comparison of reconstruction quality
# Look for: sharpness, color accuracy, preservation of details
n_samples = 5
indices = np.random.choice(len(X_test_cnn), n_samples, replace=False)

fig, axes = plt.subplots(4, n_samples, figsize=(15, 10))

for i, idx in enumerate(indices):
    # Original image (ground truth)
    axes[0, i].imshow(X_test_cnn[idx].squeeze(), cmap='gray')
    axes[0, i].set_title('Original')
    axes[0, i].axis('off')

    # Deep Conv AE reconstruction
    axes[1, i].imshow(X_test_reconstructed_conv[idx].squeeze(), cmap='gray')
    axes[1, i].set_title('Deep Conv AE')
    axes[1, i].axis('off')

    # Single Layer AE reconstruction
    axes[2, i].imshow(X_test_reconstructed_single[idx].squeeze(), cmap='gray')
    axes[2, i].set_title('Single Layer AE')
    axes[2, i].axis('off')

    # Three Layer AE reconstruction
    axes[3, i].imshow(X_test_reconstructed_three[idx].squeeze(), cmap='gray')
    axes[3, i].set_title('3-Layer AE')
    axes[3, i].axis('off')

plt.suptitle('Reconstruction Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Comparison of Reconstruction Errors

In [ ]:
# Quantitative comparison of all three autoencoders
# Lower MSE/RMSE indicates better reconstruction
# Deep conv AE should have lowest error due to spatial inductive bias
comparison_ae = {
    'Model': [
        'Deep Convolutional AE',
        'Single Layer AE (Sigmoid)',
        '3-Layer AE (Sigmoid)'
    ],
    'Architecture': [
        f'Conv layers → {K} latent',
        f'{K} nodes, Sigmoid',
        f'{nodes_per_layer}→{nodes_per_layer*2}→{nodes_per_layer*3} nodes'
    ],
    'MSE': [
        f"{reconstruction_error_conv:.6f}",
        f"{reconstruction_error_single:.6f}",
        f"{reconstruction_error_three:.6f}"
    ],
    'RMSE': [
        f"{np.sqrt(reconstruction_error_conv):.6f}",
        f"{np.sqrt(reconstruction_error_single):.6f}",
        f"{np.sqrt(reconstruction_error_three):.6f}"
    ]
}

comparison_ae_df = pd.DataFrame(comparison_ae)
print("\n" + "="*80)
print("COMPARISON: Autoencoder Reconstruction Errors (Test Set)")
print("="*80)
print(comparison_ae_df.to_string(index=False))
print("="*80)

In [ ]:
# Bar plot for visual comparison of reconstruction errors
# Makes it easy to see relative performance at a glance
models = ['Deep Conv AE', 'Single Layer AE', '3-Layer AE']
mse_values = [reconstruction_error_conv, reconstruction_error_single, reconstruction_error_three]

plt.figure(figsize=(10, 6))
bars = plt.bar(models, mse_values, color=['#2E86AB', '#A23B72', '#F18F01'])
plt.ylabel('Mean Squared Error (MSE)', fontsize=12)
plt.title('Reconstruction Error Comparison (Test Set)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on top of bars for exact comparison
for bar, val in zip(bars, mse_values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.6f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Task 4: MNIST 7-Segment LED Display Classifier [3 marks]

### Objectives:
1. Train deep convolutional autoencoder on MNIST
2. Extract latent features
3. Create 7-segment LED mapping for digits (7 outputs)
4. Train MLP classifier on extracted features
5. Generate confusion matrix

In [ ]:
# Load MNIST dataset - handwritten digits 0-9
# MNIST: 28x28 grayscale images, 60k training + 10k test
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()

# Normalize to [0, 1] range for neural network training
# Add channel dimension for CNN compatibility
X_train_mnist = X_train_mnist.astype('float32') / 255.0
X_test_mnist = X_test_mnist.astype('float32') / 255.0
X_train_mnist = X_train_mnist.reshape(-1, 28, 28, 1)
X_test_mnist = X_test_mnist.reshape(-1, 28, 28, 1)

print(f"MNIST Dataset:")
print(f"Training: {X_train_mnist.shape}, Labels: {y_train_mnist.shape}")
print(f"Test: {X_test_mnist.shape}, Labels: {y_test_mnist.shape}")

In [ ]:
# Visualize sample MNIST images to understand the data
# Display first 10 images (one of each digit ideally)
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(X_train_mnist[i].squeeze(), cmap='gray')
    axes[i].set_title(f"Digit: {y_train_mnist[i]}")
    axes[i].axis('off')

plt.suptitle('Sample MNIST Digits')
plt.tight_layout()
plt.show()

### Build and Train Deep Convolutional Autoencoder for MNIST

In [ ]:
# Deep Convolutional Autoencoder for MNIST (28x28 images)# Similar architecture to CIFAR10 but adapted for smaller images# Encoder and decoder are separated for easier feature extraction laterdef build_mnist_conv_autoencoder(latent_dim=64):    """    Architecture Overview:        ENCODER:  28x28x1 -> 14x14x32 -> 7x7x64 -> 3136 -> 64        DECODER:  64 -> 3136 -> 7x7x64 -> 14x14x64 -> 28x28x32 -> 28x28x1        Key difference from CIFAR10 autoencoder:        - Smaller input (28x28 vs 32x32)        - Fewer downsampling steps (2 vs 3)        - Separated encoder/decoder for feature extraction    """        # ========================================================================    # ENCODER: Extract features from digit images    # ========================================================================    encoder_input = layers.Input(shape=(28, 28, 1))        # Layer 1: Extract low-level features (edges, strokes)    #   Conv2D: 32 filters learn basic digit patterns    #   - activation='relu': non-linearity for pattern detection    #   - padding='same': maintains 28x28 spatial dimensions    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)        # MaxPooling2D: Spatial downsampling    #   - pool_size=(2,2): reduces dimensions by half    #   - Result: 28x28 -> 14x14 (first compression)    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 28x28 -> 14x14        # Layer 2: Extract higher-level features (digit shapes, curves)    #   64 filters: increased capacity for complex patterns    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)        # MaxPooling2D: Further spatial downsampling    #   Result: 14x14 -> 7x7 (second compression)    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 14x14 -> 7x7        # Flatten: Convert spatial features to vector    #   - Input: 7x7x64 spatial feature maps    #   - Output: 3136-dimensional vector (7 * 7 * 64 = 3136)    x = layers.Flatten()(x)        # Dense: Compress to latent representation    #   - 3136 -> 64: Final bottleneck    #   - This 64-dim vector encodes digit identity    #   - Will be used as input to 7-segment classifier    latent = layers.Dense(latent_dim, activation='relu', name='latent')(x)        # Create encoder model for feature extraction    #   - Separate model allows easy feature extraction    #   - Input: 28x28x1 image, Output: 64-dim latent vector    encoder = models.Model(encoder_input, latent, name='encoder')        # ========================================================================    # DECODER: Reconstruct images from latent features    # ========================================================================    decoder_input = layers.Input(shape=(latent_dim,))        # Dense: Expand from latent space to spatial representation size    #   - 64 -> 3136: First expansion    #   - Prepares for reshaping to 7x7x64    x = layers.Dense(7 * 7 * 64, activation='relu')(decoder_input)        # Reshape: Convert 1D vector to 3D spatial format    #   - 3136 -> 7x7x64: Restore spatial structure    x = layers.Reshape((7, 7, 64))(x)        # Conv2DTranspose: Learnable upsampling    #   - strides=2: doubles spatial dimensions (7x7 -> 14x14)    #   - 64 filters: maintains feature richness    x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same')(x)  # 7x7 -> 14x14        # Conv2DTranspose: Second upsampling step    #   - strides=2: doubles again (14x14 -> 28x28)    #   - 32 filters: reduces feature count toward output    x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation='relu', padding='same')(x)  # 14x14 -> 28x28        # Output layer: Reconstruct single-channel grayscale image    #   - filters=1: single channel (grayscale)    #   - activation='sigmoid': outputs [0, 1] range    #   - padding='same': maintains 28x28 dimensions    decoder_output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)        # Create decoder model    decoder = models.Model(decoder_input, decoder_output, name='decoder')        # Full autoencoder: combines encoder and decoder    #   - Input: 28x28x1 image    #   - Output: 28x28x1 reconstructed image    #   - Training: unsupervised reconstruction task    autoencoder = models.Model(encoder_input, decoder(encoder(encoder_input)),                               name='mnist_autoencoder')        return autoencoder, encoder, decodermnist_ae, mnist_encoder, mnist_decoder = build_mnist_conv_autoencoder(latent_dim=64)mnist_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])print("\nMNIST Convolutional Autoencoder:")mnist_ae.summary()

In [ ]:
# Train MNIST autoencoder
# Goal: Learn compact 64-dim representation of digits
# These features will be used for 7-segment classification
print("Training MNIST Convolutional Autoencoder...")

history_mnist = mnist_ae.fit(
    X_train_mnist, X_train_mnist,  # Unsupervised learning
    epochs=20,  # Fewer epochs - MNIST is easier than CIFAR10
    batch_size=128,
    shuffle=True,
    validation_data=(X_test_mnist, X_test_mnist),
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Visualize MNIST reconstructions to verify autoencoder quality
# Good reconstructions indicate useful learned features
n_samples = 10
X_mnist_reconstructed = mnist_ae.predict(X_test_mnist[:n_samples])

fig, axes = plt.subplots(2, n_samples, figsize=(15, 3))

for i in range(n_samples):
    # Original digit image
    axes[0, i].imshow(X_test_mnist[i].squeeze(), cmap='gray')
    axes[0, i].set_title(f'Digit: {y_test_mnist[i]}')
    axes[0, i].axis('off')

    # Reconstructed image
    axes[1, i].imshow(X_mnist_reconstructed[i].squeeze(), cmap='gray')
    axes[1, i].set_title('Reconstructed')
    axes[1, i].axis('off')

plt.suptitle('MNIST: Original vs Reconstructed', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Create 7-Segment LED Mapping

7-segment display layout:
```
     0
   -----
  |     |
1 |     | 2
  |  3  |
   -----
  |     |
4 |     | 5
  |     |
   -----
     6
```

Segments: [top, top-left, top-right, middle, bottom-left, bottom-right, bottom]

In [ ]:
# 7-segment LED mapping for digits 0-9
# Each digit is represented by which segments are ON (1) or OFF (0)
# Format: [top, top-left, top-right, middle, bottom-left, bottom-right, bottom]
#
# Visual reference:
#      top(0)
#   ---------
#   |       |
# TL|       |TR
# (1)       (2)
#   | mid(3)|
#   ---------
#   |       |
# BL|       |BR
# (4)       (5)
#   | bot(6)|
#   ---------
seven_segment_map = {
    0: [1, 1, 1, 0, 1, 1, 1],  # 0: all except middle
    1: [0, 0, 1, 0, 0, 1, 0],  # 1: right side only
    2: [1, 0, 1, 1, 1, 0, 1],  # 2: classic "2" shape
    3: [1, 0, 1, 1, 0, 1, 1],  # 3: classic "3" shape
    4: [0, 1, 1, 1, 0, 1, 0],  # 4: top-left, middle, right
    5: [1, 1, 0, 1, 0, 1, 1],  # 5: classic "S" shape
    6: [1, 1, 0, 1, 1, 1, 1],  # 6: all except top-right
    7: [1, 0, 1, 0, 0, 1, 0],  # 7: top and right side
    8: [1, 1, 1, 1, 1, 1, 1],  # 8: all segments on
    9: [1, 1, 1, 1, 0, 1, 1],  # 9: all except bottom-left
}

# Convert digit labels to 7-segment binary representations
# This transforms classification problem to multi-label binary classification
y_train_7seg = np.array([seven_segment_map[digit] for digit in y_train_mnist])
y_test_7seg = np.array([seven_segment_map[digit] for digit in y_test_mnist])

print(f"7-Segment Labels:")
print(f"Training: {y_train_7seg.shape}")  # (60000, 7)
print(f"Test: {y_test_7seg.shape}")        # (10000, 7)

# Display mapping for verification
print("\n7-Segment Mapping:")
for digit, segments in seven_segment_map.items():
    print(f"Digit {digit}: {segments}")

In [ ]:
# Function to draw 7-segment LED display
# Used for visualizing predicted and ground truth segment patterns
def draw_7segment(segments, ax, title):
    """
    Draw a 7-segment LED display representation

    Args:
        segments: List of 7 binary values (1=ON, 0=OFF)
        ax: Matplotlib axis to draw on
        title: Title for the plot
    """
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 5)
    ax.axis('off')
    ax.set_aspect('equal')

    # Define line segments for each LED segment
    # [top, top-left, top-right, middle, bottom-left, bottom-right, bottom]
    segment_lines = [
        [(0.5, 4.5), (2.5, 4.5)],  # top horizontal
        [(0.5, 4.5), (0.5, 2.5)],  # top-left vertical
        [(2.5, 4.5), (2.5, 2.5)],  # top-right vertical
        [(0.5, 2.5), (2.5, 2.5)],  # middle horizontal
        [(0.5, 2.5), (0.5, 0.5)],  # bottom-left vertical
        [(2.5, 2.5), (2.5, 0.5)],  # bottom-right vertical
        [(0.5, 0.5), (2.5, 0.5)],  # bottom horizontal
    ]

    # Draw each segment based on ON/OFF state
    for seg, line in zip(segments, segment_lines):
        color = 'black' if seg == 1 else 'lightgray'  # ON = black, OFF = gray
        linewidth = 8 if seg == 1 else 2  # Thicker when ON
        ax.plot([line[0][0], line[1][0]], [line[0][1], line[1][1]],
                color=color, linewidth=linewidth)

    ax.set_title(title, fontsize=10)

# Visualize 7-segment representation for all 10 digits
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
axes = axes.ravel()

for digit in range(10):
    draw_7segment(seven_segment_map[digit], axes[digit], f'Digit {digit}')

plt.suptitle('7-Segment LED Display Representations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Extract Latent Features and Train MLP Classifier

In [ ]:
# Extract latent features from trained autoencoder
# These 64-dim features will be input to the 7-segment classifier
# Using encoder (not full autoencoder) for feature extraction
print("Extracting latent features from MNIST autoencoder...")
X_train_latent = mnist_encoder.predict(X_train_mnist)
X_test_latent = mnist_encoder.predict(X_test_mnist)

print(f"Latent features shape:")
print(f"Training: {X_train_latent.shape}")  # (60000, 64)
print(f"Test: {X_test_latent.shape}")        # (10000, 64)

In [ ]:
# Build MLP classifier to predict 7-segment LED representation# Input: Latent features from autoencoder (64-dim)# Output: 7 binary values (one for each LED segment)def build_7segment_classifier(input_dim):    """    Architecture Overview:        INPUT:  64-dim latent features (from MNIST autoencoder)        HIDDEN: 64 -> 128 -> 64 (with dropout regularization)        OUTPUT: 7 binary predictions (one per LED segment)        Key design choices:        - Multi-label classification (7 independent outputs)        - Sigmoid output (each segment probability [0, 1])        - Dropout for regularization (prevent overfitting)        - Binary crossentropy loss (suitable for multi-label)    """    model = models.Sequential([        layers.Input(shape=(input_dim,)),                # HIDDEN LAYER 1: Learn complex patterns from latent features        #   - Input: 64-dim latent features (digit representations)        #   - Output: 128 neurons        #   - activation='relu': non-linear transformation        #   - Purpose: Expand representation to capture digit-to-segment mappings        layers.Dense(128, activation='relu'),                # Dropout: Regularization to prevent overfitting        #   - rate=0.3: randomly drops 30% of neurons during training        #   - Prevents co-adaptation of neurons        #   - Improves generalization to test set        layers.Dropout(0.3),                # HIDDEN LAYER 2: Further refinement        #   - Input: 128 neurons        #   - Output: 64 neurons        #   - Compresses back to prepare for output layer        layers.Dense(64, activation='relu'),        layers.Dropout(0.3),                # OUTPUT LAYER: 7 independent binary predictions        #   - Input: 64 neurons        #   - Output: 7 neurons (one per LED segment)        #   - activation='sigmoid': each output in [0, 1] range        #   - Each output represents probability of segment being ON        #           # Segment mapping:        #   Output[0] = top segment        #   Output[1] = top-left segment        #   Output[2] = top-right segment        #   Output[3] = middle segment        #   Output[4] = bottom-left segment        #   Output[5] = bottom-right segment        #   Output[6] = bottom segment        layers.Dense(7, activation='sigmoid')  # 7 outputs for 7 segments    ], name='7segment_classifier')        return modelclassifier_7seg = build_7segment_classifier(input_dim=X_train_latent.shape[1])# Compile with appropriate loss and metrics# Binary crossentropy: treats each of 7 outputs as independent binary classification#   - Loss calculated separately for each segment#   - Suitable for multi-label classification (segments can be ON/OFF independently)# Accuracy: measures how often predicted segment matches true segment (across all 7)classifier_7seg.compile(    optimizer='adam',    loss='binary_crossentropy',  # Binary loss for each segment    metrics=['accuracy']  # Accuracy across all segments)print("\n7-Segment LED Classifier:")classifier_7seg.summary()

In [ ]:
# Train 7-segment classifier on extracted latent features
# Goal: Map digit features to 7 binary segment outputs
# This is a novel intermediate representation task
print("Training 7-Segment LED Classifier...")

history_7seg = classifier_7seg.fit(
    X_train_latent, y_train_7seg,  # Input: features, Output: 7 segments
    epochs=20,
    batch_size=128,
    validation_data=(X_test_latent, y_test_7seg),
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Plot training history for 7-segment classifier
# Monitor convergence and check for overfitting
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Binary crossentropy loss (lower is better)
axes[0].plot(history_7seg.history['loss'], label='Training Loss')
axes[0].plot(history_7seg.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Binary Crossentropy Loss')
axes[0].set_title('7-Segment Classifier - Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy across all 7 segments (higher is better)
axes[1].plot(history_7seg.history['accuracy'], label='Training Accuracy')
axes[1].plot(history_7seg.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('7-Segment Classifier - Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Generate Predictions and Confusion Matrix

In [ ]:
# Generate 7-segment predictions for test set
# Convert probabilities to binary (threshold at 0.5)
y_pred_7seg_prob = classifier_7seg.predict(X_test_latent)
y_pred_7seg = (y_pred_7seg_prob > 0.5).astype(int)  # Binarize predictions

print(f"Prediction shape: {y_pred_7seg.shape}")  # (10000, 7)
print(f"\nSample predictions (first 5):")
for i in range(5):
    print(f"True digit: {y_test_mnist[i]}, Predicted segments: {y_pred_7seg[i]}, True segments: {y_test_7seg[i]}")

In [ ]:
# Convert 7-segment predictions back to digit labels for evaluation
def segments_to_digit(segments):
    """
    Convert 7-segment binary representation back to digit (0-9)

    Args:
        segments: Array of 7 binary values [top, top-left, top-right, middle,
                  bottom-left, bottom-right, bottom]

    Returns:
        digit (0-9) if pattern matches, -1 if unknown pattern
    """
    segments_tuple = tuple(segments)
    for digit, seg_map in seven_segment_map.items():
        if tuple(seg_map) == segments_tuple:
            return digit
    return -1  # Unknown pattern (prediction doesn't match any valid digit)

# Convert all predictions back to digit labels
y_pred_digits = np.array([segments_to_digit(seg) for seg in y_pred_7seg])

# Calculate overall accuracy
correct_predictions = np.sum(y_pred_digits == y_test_mnist)
total_predictions = len(y_test_mnist)
accuracy = correct_predictions / total_predictions

print(f"\n7-Segment LED Classifier Results:")
print(f"Correct predictions: {correct_predictions} / {total_predictions}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Unknown patterns (-1): {np.sum(y_pred_digits == -1)}")
print(f"Note: Unknown patterns occur when predicted segment combination doesn't match any digit")

In [ ]:
# Generate and visualize confusion matrix
# Shows how often each digit is classified as each other digit
# Diagonal elements = correct classifications
# Off-diagonal elements = misclassifications
cm = confusion_matrix(y_test_mnist, y_pred_digits)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(range(10)) + ['Unknown'],
            yticklabels=list(range(10)),
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Digit', fontsize=12)
plt.ylabel('True Digit', fontsize=12)
plt.title('Confusion Matrix - 7-Segment LED Classifier', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Detailed classification report with precision, recall, F1-score per class
print("\nClassification Report:")
print(classification_report(y_test_mnist, y_pred_digits,
                          target_names=[str(i) for i in range(10)]))

In [ ]:
# Visualize predictions: Original image → Predicted 7-seg → True 7-seg
# This shows the complete pipeline: image → features → segments
n_samples = 10
indices = np.random.choice(len(X_test_mnist), n_samples, replace=False)

fig, axes = plt.subplots(3, n_samples, figsize=(20, 6))

for i, idx in enumerate(indices):
    # Row 1: Original MNIST image
    axes[0, i].imshow(X_test_mnist[idx].squeeze(), cmap='gray')
    axes[0, i].set_title(f'True: {y_test_mnist[idx]}')
    axes[0, i].axis('off')

    # Row 2: Predicted 7-segment display
    draw_7segment(y_pred_7seg[idx], axes[1, i], 'Predicted')

    # Row 3: Ground truth 7-segment display
    draw_7segment(y_test_7seg[idx], axes[2, i], f'True')

# Add row labels
axes[0, 0].set_ylabel('Original', fontsize=12)
axes[1, 0].set_ylabel('Predicted', fontsize=12)
axes[2, 0].set_ylabel('Ground Truth', fontsize=12)

plt.suptitle('7-Segment LED Classifier: Predictions vs Ground Truth',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Summary and Conclusions

## Task 1: PCA-based Classification
- Identified K components retaining 95% variance
- Both Standard and Randomized PCA achieved similar performance
- Logistic regression on PCA features provided baseline classification

## Task 2: Linear Autoencoder vs PCA
- Single-layer linear autoencoder with tied weights learns similar features to PCA
- Weight vectors capture principal directions in data
- Visual comparison shows structural similarity between eigenvectors and learned weights

## Task 3: Deep Convolutional Autoencoders
- Deep convolutional architecture significantly outperforms shallow autoencoders
- Hierarchical feature learning enables better reconstruction
- Distributing nodes across multiple layers (3-layer) provides intermediate performance

## Task 4: MNIST 7-Segment LED Classifier
- Successfully trained autoencoder to extract MNIST features
- MLP classifier learned to map latent features to 7-segment representations
- Confusion matrix reveals digit-specific performance patterns
- Novel task demonstrating practical application of learned representations